**Soovitatav on käivitada kõik järgnev Google Colab'is. Teistel platvormidel ei saa garanteerida, et kõik toimib nii nagu peaks.**

In [ ]:
# Colabi jaoks
from google.colab import drive
from google.colab import runtime
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Lõputöö/Bakalaureus') # Muutke oma Drive asukohaks

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
# Importimised
import re
import time
import random
import csv
from collections import defaultdict

## Andmete kogumine, analüüs ja statistika

Selles peatükis kogutakse andmeid leksikonist kasutades regulaaravalidisi ning koostatakse sõnaliigi ja muuttüübi sagedustabeleid.

### Andmete kogumine

Järgmise koodiga uuritakse palju kirjeid on leksikonis.

In [ ]:
# Kirjete arv
with open('fs_lex.txt', 'r') as fs_lex:
  ridade_arv = 0
  fs_lex.readline() # Esimene rida on tühi rida
  for rida in fs_lex:
    ridade_arv += 1
  print(ridade_arv) # 73831

73831


#### Täiendatud versioon

In [ ]:
# Testimine
puhastatud = re.sub(r'[^a-zA-Z0-9-\/\\=ÕÄÖÜŠŽõäöüšž]', '', 'And?orra=la=Vella')
if '=' in puhastatud: # Mitmesõnaline
  puhastatud = puhastatud.replace('=', ' ')

if puhastatud[0].isupper():
  if '-' not in puhastatud:
    sõnasid = puhastatud.split()
    print(sõnasid)
    puhastatud = ""
    for s in sõnasid:
      if s[0].isupper():
        puhastatud += s[0] + s[1:].lower() + " "
      else:
        puhastatud += s + " "
  else:
    sõnasid = puhastatud.split("-")
    print(sõnasid)
    puhastatud = ""
    for s in sõnasid:
      if s[0].isupper():
        puhastatud += s[0] + s[1:].lower() + "-"
      else:
        puhastatud += s + " "
  puhastatud = puhastatud.strip("-")

else:
  puhastatud = puhastatud.lower()

print(puhastatud)

['Andorra', 'la', 'Vella']
Andorra la Vella 


In [ ]:
testkirjed = ["lisav:0919|**s<un]d_l<iit[MA s<un]d_l<iit[A s<un]d_liida[B s<un]d_liide[TUD s<un]d_l<iit[MATA s<un]d_l<iit[EV s<un]d_l<iit[IS s<un]d_l<iit[KU s<un]d_liida[ME s<un]d_liide[TAKSE s<un]d_l<iit[SIN s<un]d_l<iit[SIME s<un]d_l<iit[NUD !V.34!",
              "sage:0905|kahe_k<ümnene kahe_k<ümnese kahe_k<ümnes[T!\SAm\.12^I^+7!&&!\Am\.10+7!"]
sõnaliigi_regex = r'\\(\w)\w{0,5}\\|\!(\w)\w{0,5}\.'
test_list = []
for testkirje in testkirjed:
  ss = re.findall(sõnaliigi_regex, testkirje)
  if ss:
    for i in range(len(ss)):
      for s in ss[i]:
        if s:
          ss[i] = s
          break
      if ss[i] == 'B': # Sisemiselt kasutatakse sellist sõnaliiki, et määrata omadussõna, millest kesk- või ülivõrde moodustamine ei käi reeglipäraselt, nt. lahja
        ss[i] = 'A' # Selle asemele omadussõna algvõrde
    test_list.append(ss)
print(test_list)

[['V'], ['S', 'A']]


Järgmine kood kogub igast kirjest algvormi, muuttüübi ja sõnaliigi kasutades regulaaravaldisi.

In [ ]:
# Regulaaravaldised
sõna_regex = r'\|(.*?)!'
sõnaliigi_regex = r'\\(\w)\w{0,5}\\|\!(\w)\w{0,5}\.'
muuttüübi_regex = r'\.(\d{1,2})'

# Järjendid erinevate andmete kogumiseks
sõnad = []
sõnaliigid = []
muuttüübid = []

# Erandite loendamine. Sümbolid ** määravad, et tegu on erindiga
count_erandid = 0

# Välistab sõnade asemele pandud muud tähised algvormi otsimisel
keelud = ['$', '--']

# Leksikoni avamine
with open('fs_lex.txt', 'r') as fs_lex:
  # Andmete kogumine
  fs_lex.readline() # Esimene rida on tühi
  for rida in fs_lex:
      #time.sleep(0.125)
      #display(rida)
      #info = rida.split('|')[1].split('!')

      if re.search(r'.*(\*\*).*', rida): # Erandid
        count_erandid += 1

      # Sõna kättesaamine
      #sõnu = info[0].split(' ')
      sõnu = re.findall(sõna_regex, rida)
      if sõnu:
        sõnu = sõnu[0].split()
        for s in sõnu: # Võtab esimese seda esindavat muutevormi, mis on olemas ning välistab muud tähised
          if s:
            järgmine = False
            for keeld in keelud:
              if keeld in s:
                järgmine = True
                break
            if järgmine:
              #print(s)
              continue
            sõna = s
            break
      else:
        continue

      # Sõnaliigi kättesaamine
      sõnaliik = re.findall(sõnaliigi_regex, rida)

      if sõnaliik:
        for i in range(len(sõnaliik)):
          for sl in sõnaliik[i]: # Regexi disjunktsioon | annab kaks vastet, millest üks on tühi
            if sl:
              sõnaliik[i] = sl # Võtab mittetühja vaste
              break
          if sõnaliik[i] == 'B': # Sisemiselt kasutatakse sellist sõnaliiki, et määrata omadussõna, millest kesk- või ülivõrde moodustamine ei käi reeglipäraselt, nt. lahja
            sõnaliik[i] = 'A' # Selle asemele omadussõna algvõrde
        sõnaliigid.append(sõnaliik)
      else:
        print(rida)

      # Muuttüübi kättesaamine
      muuttüüp = re.findall(muuttüübi_regex, rida)
      muuttüübid.append(muuttüüp)

      # Sõnade puhastamine -- tegusõnadel näiteks MA tähis väiketähtedeks
      puhastatud = re.sub(r'[\+\<\?\]\~\_\[\(\)\&\#\’\*\,\$\*]', '', sõna)
      if '=' in puhastatud: # Sümbol = tähistab, et tegu on sõnaühendiga
        puhastatud = puhastatud.replace('=', ' ')

      if puhastatud[0].isupper(): # Kui sõna on suure algustähega
        if '-' not in puhastatud: # Sõnas pole sidekriipsu
          sõnasid = puhastatud.split()
          puhastatud = ""
          for s in sõnasid: # Kui sõna sõnaühendis on suure algustähega, siis jäta suur algustäht sisse
            if s[0].isupper():
              puhastatud += s[0] + s[1:].lower() + " "
            else:
              puhastatud += s + " "
          puhastatud = puhastatud.strip()

        else: # Sõna on sidekriipsuga
          sõnasid = puhastatud.split("-")
          puhastatud = ""
          for s in sõnasid:
            if s[0].isupper(): # Kui sõna sõnaühendis on suure algustähega, siis jäta suur algustäht sisse
              puhastatud += s[0] + s[1:].lower() + "-"
            else:
              puhastatud += s + " "
        puhastatud = puhastatud.strip("-")

      else: # Sõna pole suure algustähega, mistõttu muuda kõik väiketähtedeks
        puhastatud = puhastatud.lower()

      if puhastatud: # Kui puhastatud, siis lisa sõna järjendisse
        sõnad.append([puhastatud])
      #print(puhastatud, sõnaliik, muuttüüp)

# Eitussõnad "ei" ja "ära"
#sõnad.extend([["ei"], ["ära"]])
#sõnaliigid.extend([[], []])
#muuttüübid.extend([[], []])

Järgmise koodiga kontrollitakse andmete kogumist, kas kõik kirjed saadi kätte, kas jäi mingi kirje vahele. Lisaks väljastatakse esimesed ja viimased 20 kirjet.

In [ ]:
print(f"Kirjete arv: {ridade_arv}")
print(f"Sõnaliike kokku: {len(sõnaliigid)}")
print(f"Muuttüüpe kokku: {len(muuttüübid)}")
print(f"Sõnu kokku: {len(sõnad)} \t Erinevus tuleneb muutevormide ära ja ei eemaldamises")
print(f"Millest erandeid: {count_erandid}")
print(sõnad[:20])
print(muuttüübid[:20])
print(sõnaliigid[:20])
print(sõnad[-20:])
print(muuttüübid[-20:])
print(sõnaliigid[-20:])

for sõna, sõnaliik, muuttüüp in zip(sõnad, sõnaliigid, muuttüübid):
  if sõnaliik == ['S', 'A']:
    print(sõna, sõnaliik, muuttüüp)

Kirjete arv: 73831
Sõnaliike kokku: 73829
Muuttüüpe kokku: 73829
Sõnu kokku: 73829 	 Erinevus tuleneb muutevormide ära ja ei eemaldamises
Millest erandeid: 2932
[['1/16-finaal'], ['1/2-finaal'], ['1/4-finaal'], ['1/8-finaal'], ['aabe'], ['Aabel'], ['aabits'], ['aabitsajünger'], ['aabitsajüts'], ['aabitsalik'], ['aabitsaraamat'], ['aabitsatarkus'], ['aabitsatõde'], ['Aabram'], ['Aachen'], ['Aadam'], ['aadamaõun'], ['aadamaülikond'], ['aadamaülikonnas'], ['aade']]
[['22'], ['22'], ['22'], ['22'], ['6'], ['2'], ['2'], ['2'], ['22'], ['25'], ['2'], ['11', '9'], ['18'], ['2'], ['2'], ['2'], ['22'], ['22'], [], ['6']]
[['S'], ['S'], ['S'], ['S'], ['S'], ['H'], ['S'], ['S'], ['S'], ['A'], ['S'], ['S', 'S'], ['S'], ['H'], ['H'], ['H'], ['S'], ['S'], ['A'], ['S']]
[['üürivõlg'], ['üürnik'], ['üür'], ['Xbox'], ['Xian'], ['x-jalg'], ['x-suurus'], ['Yamoussoukro'], ['Yangon'], ['Yaounde'], ['Yaoundé'], ['Yaren'], ['Yoko-Gumi'], ['Yokohama'], ['yokozuna'], ['Yomi'], ['yo-on'], ['Yorkshire'], ['York

Järgmises koodis väljastatakse muuttüüp 0 sõnu.

In [ ]:
# Muuttüüp 0 kirjed
i = 0 # indeks ehk rida
count = 0 # loendamiseks
for sõna, sõnaliik, muuttüüp in zip(sõnad, sõnaliigid, muuttüübid):
  i+=1
  for m in muuttüüp:
    if int(m) == 0:
      count+=1
      print("0 muuttüübiga", end = " ")
      print(i, sõna, sõnaliik, muuttüüp)
print(f"Kokku on muuttüüp 0 sõnu: {count}")

0 muuttüübiga 843 ['ajalehed-ajakirjad'] ['S'] ['0']
0 muuttüübiga 6702 ['eikeegi'] ['P'] ['0']
0 muuttüübiga 6717 ['eimiski'] ['P'] ['0']
0 muuttüübiga 7026 ['elamine-olemine'] ['S'] ['0']
0 muuttüübiga 7471 ['ematu-isatu'] ['S'] ['0']
0 muuttüübiga 7479 ['emb-kumb'] ['P'] ['0']
0 muuttüübiga 14958 ['ise'] ['P'] ['0']
0 muuttüübiga 14959 ['iseenese'] ['P'] ['0']
0 muuttüübiga 15187 ['istumine-astumine'] ['S'] ['0']
0 muuttüübiga 19951 ['keegi'] ['P'] ['0']
0 muuttüübiga 20529 ['kes'] ['P'] ['0']
0 muuttüübiga 20739 ['kibin-kabin'] ['S'] ['0']
0 muuttüübiga 20781 ['kigin-kägin'] ['S'] ['0']
0 muuttüübiga 20802 ['kihin-kahin'] ['S'] ['0']
0 muuttüübiga 21156 ['kilin-kolin'] ['S'] ['0']
0 muuttüübiga 21157 ['kilin-kõlin'] ['S'] ['0']
0 muuttüübiga 21272 ['kimpsud-kompsud'] ['S'] ['0']
0 muuttüübiga 21603 ['kirin-kärin'] ['S'] ['0']
0 muuttüübiga 22196 ['klibin-klobin'] ['S'] ['0']
0 muuttüübiga 24389 ['kopsud-maksad'] ['S'] ['0']
0 muuttüübiga 25880 ['kumbki'] ['P'] ['0']
0 muuttüübiga 2

### Analüüs ja statistika

Selles peatükis koostatakse sõnaliigi ja muuttüübi sagedustabeleid.

Järgmine info on võetud [Leksikoni kirjeldusest](https://github.com/Filosoft/vabamorf/blob/master/doc/morfi_leksikoni_kirjeldus.md):

#### Lingvistiliselt sisukad sõnaliigid leksikonis on järgmised:
 * `A` omadussõna - algvõrre (adjektiiv - positiiv), nii käänduvad kui käändumatud, nt kallis või eht
 * `C` omadussõna - keskvõrre (adjektiiv - komparatiiv), nt laiem
 * `D` määrsõna (adverb), nt kõrvuti
 * `G` genitiivatribuut (käändumatu omadussõna), nt balti
 * `H` pärisnimi, nt Edgar
 * `I` hüüdsõna (interjektsioon), nt tere
 * `J` sidesõna (konjunktsioon), nt ja
 * `K` kaassõna (pre/postpositsioon), nt kaudu
 * `N` põhiarvsõna (kardinaalnumeraal), nt kaks
 * `O` järgarvsõna (ordinaalnumeraal), nt teine
 * `P` asesõna (pronoomen), nt see
 * `S` nimisõna (substantiiv), nt asi
 * `U` omadussõna - ülivõrre (adjektiiv - superlatiiv), nt pikim
 * `V` tegusõna (verb), nt lugema
 * `X` verbi juurde kuuluv sõna, millel eraldi sõnaliigi tähistus puudub, nt plehku

#### Morf. analüsaator väljastab algoritmiliselt analüüsitavatele tekstielementidele ka selliseid 2 liiki, mida sõnastikus ei esine:

* `Y` lühend, nt USA
* `Z` lausemärk, nt `-`, `/`, `...`

#### Sisemiselt kasutatakse veel järgmisi sõnaliike:

* `B` omadussõna, millest kesk- või ülivõrde moodustamine ei käi reeglipäraselt, nt. lahja
* `W` deverbaal, s.t. sõna on tuletatud verbist, nt. toitev, jooks; esineb alati koos liigiga A või S

#### Sõnaliigi väljal võib peale sõnaliigi olla ka mõni järgmine tunnus:

* `t` tabusõna (nt perse)
* `s` ainult spelleris lubatav üksiksõna (nt Aires, Janeiro)
* `e` ainult tesauruses sellisena omaette sõnana esitatav sõna (nt aukudega (=auklik))
* `m` liitsõna koosseisu mittesobiv sõna (nt viieaastane)
* `n` liitsõna lõppu mittesobiv sõna (nt abt, bai, utt)


#### Sõnaliigi sagedused

Järgmine kood loeb kokku sõnaliike järgmiselt:
* kui sõnal on ainult üks sõnaliik, siis vastava sõnaliigi loenduri väärtus tõstetakse ühe võrra
* kui sõnal on mitu sõnaliiki, siis võetakse esimene sõnaliik ja selle loenduri väärtus tõstetakse ühe võrra

Lõpus väljastatakse sagedustabel.

In [ ]:
# Sõnaliigi sagedused (ainult esimene sõnaliik mitme sõnaliigiga sõnast)
sõnaliigid_freq = defaultdict(lambda: 0)
for sõnaliik_list in sõnaliigid:
  if len(sõnaliik_list) == 1: # Ühe sõnaliigiga
    sõnaliigid_freq[sõnaliik_list[0]] += 1
  else: # Mitme sõnaliigiga
    sõnaliigid_freq[sõnaliik_list[0]] += 1

sõnaliigid_freq = sorted(sõnaliigid_freq.items(), key=lambda item: item[1], reverse=True)
summa, summa_protsent = 0, 0
for item, freq in sõnaliigid_freq:
  print(f"{item}: {freq} ({round(freq / len(sõnaliigid) * 100, 3)}%)")
  summa += freq
  summa_protsent += freq / len(sõnaliigid) * 100
print(f"Summa: {summa} ({round(summa_protsent, 3)}%)")

S: 40625 (55.026%)
V: 10724 (14.525%)
A: 10336 (14.0%)
D: 6460 (8.75%)
H: 3777 (5.116%)
G: 584 (0.791%)
C: 414 (0.561%)
I: 336 (0.455%)
K: 204 (0.276%)
N: 105 (0.142%)
P: 102 (0.138%)
U: 70 (0.095%)
O: 48 (0.065%)
J: 23 (0.031%)
X: 21 (0.028%)
Summa: 73829 (99.999%)


Järgmine kood loeb kokku sõnaliike järgmiselt:
* kui sõnal on ainult üks sõnaliik, siis vastava sõnaliigi loenduri väärtus tõstetakse ühe võrra
* kui sõnal on mitu sõnaliiki, siis koheldakse neid kui üht uut sõnaliiki, mille loenduri väärtus tõstetakse ühe võrra

Lõpus väljastatakse sagedustabel.

In [ ]:
# Sõnaliigi sagedused
sõnaliigid_freq = defaultdict(lambda: 0)
for sõnaliik_list in sõnaliigid:
  if len(sõnaliik_list) == 1: # Ühe sõnaliigiga
    sõnaliigid_freq[sõnaliik_list[0]] += 1
  else: # Mitme sõnaliigiga
    sl = ""
    for i in range(len(sõnaliik_list)): # Sõnaliigid ühendatakse kokku
      item = sõnaliik_list[i]
      if i == len(sõnaliik_list) - 1:
        sl += item
      else:
        sl += item + ","
    sõnaliigid_freq[sl] += 1

sõnaliigid_freq = sorted(sõnaliigid_freq.items(), key=lambda item: item[1], reverse=True)
summa, summa_protsent = 0, 0
for item, freq in sõnaliigid_freq:
  print(f"{item}: {freq} ({round(freq / len(sõnaliigid) * 100, 3)}%)")
  summa += freq
  summa_protsent += freq / len(sõnaliigid) * 100
print(f"Summa: {summa} ({round(summa_protsent, 3)}%)")

S: 37141 (50.307%)
V: 10655 (14.432%)
A: 9977 (13.514%)
D: 6419 (8.694%)
H: 3763 (5.097%)
S,S: 3406 (4.613%)
G: 584 (0.791%)
C: 414 (0.561%)
A,A: 357 (0.484%)
I: 336 (0.455%)
K: 201 (0.272%)
N: 105 (0.142%)
P: 90 (0.122%)
S,I: 73 (0.099%)
U: 70 (0.095%)
V,V: 69 (0.093%)
O: 48 (0.065%)
D,K: 41 (0.056%)
J: 23 (0.031%)
X: 21 (0.028%)
H,H: 14 (0.019%)
P,P: 12 (0.016%)
S,A: 5 (0.007%)
K,D: 3 (0.004%)
A,S: 2 (0.003%)
Summa: 73829 (100.0%)


#### Muuttüüpide sagedused

Järgmine info on võetud [Leksikoni kirjeldusest](https://github.com/Filosoft/vabamorf/blob/master/doc/morfi_leksikoni_kirjeldus.md):

Muuttüüp on sama mis [Väikeses vormisõnastikus](https://www.eki.ee/teemad/tabel_tyybid.html): käändsõnadel 1-26 ja pöördsõnadel 27-38.

Koondtabel või täpsem [Väikese vormisõnastiku](https://www.eki.ee/teemad/tabel_tyybid.html) tabel

Järgmine kood loeb kokku muuttüüpe järgmiselt:
* kui sõnal on ainult üks muuttüüp, siis vastava muuttüübi loendur tõstetakse ühe võrra
* kui sõnal on mitu muuttüüpi, siis iga muuttüübi loendur tõstetakse ühe võrra

Lisaks loendatakse muutumatuid sõnu ja muuttüüpi 0, mis on Vabamorfi loodud muuttüüp väga erandlike sõnade klassifitseerimiseks

Lõpus väljastatakse sagedustabel.

In [ ]:
# Muuttüübi sagedused

print("---Mitme muuttüübiga sõnades eraldatakse muuttüübid---\n")

muuttüübid_freq = defaultdict(lambda: 0)
# Eraldi kõik
for muuttüüp_list in muuttüübid:
  if not muuttüüp_list:
    muuttüübid_freq["-"] += 1
  for item in muuttüüp_list:
    muuttüübid_freq[item] += 1 # Võtab eraldi muuttüübid mitme muuttüübiga sõnast

käändsõnad_freq = {}
pöördsõnad_freq = {}
muuttüüp_0_freq = 0
muutumatud_freq = 0
for item, freq in muuttüübid_freq.items():
  if item == "-":
    muutumatud_freq = freq
  elif int(item) == 0:
    muuttüüp_0_freq = freq
  elif int(item) <= 26:
    käändsõnad_freq[item] = freq
  else:
    pöördsõnad_freq[item] = freq

käändsõnad_freq = sorted(käändsõnad_freq.items(), key=lambda item: item[1], reverse=True)
pöördsõnad_freq = sorted(pöördsõnad_freq.items(), key=lambda item: item[1], reverse=True)

käändsõnad_sum = 0
pöördsõnad_sum = 0
summa_protsent = 0

print("Käändsõnad")
for item, freq in käändsõnad_freq:
  print(f"{item}: {freq} ({round(freq / len(muuttüübid) * 100, 4)}%)")
  käändsõnad_sum += freq
  summa_protsent += freq / len(muuttüübid) * 100
print(f"Kokku: {käändsõnad_sum} ({round(summa_protsent, 4)}%)\n")

summa_protsent = 0

print("Pöördsõnad")
for item, freq in pöördsõnad_freq:
  print(f"{item}: {freq} ({round(freq / len(muuttüübid) * 100, 4)}%)")
  pöördsõnad_sum += freq
  summa_protsent += freq / len(muuttüübid) * 100
print(f"Kokku: {pöördsõnad_sum} ({round(summa_protsent, 4)}%)\n")
print(f"Muuttüüp 0: {muuttüüp_0_freq} ({round(muuttüüp_0_freq / len(muuttüübid) * 100, 4)}%)")
print(f"Muutumatud sõnad: {muutumatud_freq} ({round(muutumatud_freq / len(muuttüübid) * 100, 4)}%)")
print(f"\nKokku: {käändsõnad_sum + pöördsõnad_sum + muuttüüp_0_freq + muutumatud_freq} ({round((käändsõnad_sum + pöördsõnad_sum + muuttüüp_0_freq + muutumatud_freq) / len(muuttüübid) * 100, 4)}%)")

---Mitme muuttüübiga sõnades eraldatakse muuttüübid---

Käändsõnad
22: 14142 (19.155%)
2: 11447 (15.505%)
11: 6352 (8.604%)
12: 5203 (7.047%)
1: 4640 (6.285%)
9: 3048 (4.128%)
16: 2722 (3.687%)
10: 2002 (2.712%)
17: 1721 (2.331%)
25: 1514 (2.051%)
26: 1256 (1.701%)
6: 1063 (1.44%)
19: 739 (1.001%)
13: 531 (0.719%)
7: 460 (0.623%)
24: 399 (0.54%)
18: 396 (0.536%)
5: 257 (0.348%)
23: 218 (0.295%)
14: 184 (0.249%)
20: 108 (0.146%)
4: 106 (0.144%)
21: 99 (0.134%)
8: 79 (0.107%)
15: 70 (0.095%)
3: 33 (0.045%)
Kokku: 58789 (79.628%)

Pöördsõnad
27: 5650 (7.653%)
28: 2670 (3.616%)
29: 722 (0.978%)
36: 551 (0.746%)
34: 463 (0.627%)
30: 214 (0.29%)
38: 169 (0.229%)
37: 153 (0.207%)
35: 112 (0.152%)
31: 38 (0.051%)
32: 37 (0.05%)
33: 11 (0.015%)
Kokku: 10790 (14.614%)

Muuttüüp 0: 54 (0.073%)
Muutumatud sõnad: 8061 (10.918%)

Kokku: 77694 (105.235%)


Järgmine kood loeb kokku muuttüüpe järgmiselt:
* kui sõnal on ainult üks muuttüüp, siis vastava muuttüübi loendur tõstetakse ühe võrra
* kui sõnal on mitu muuttüüpi, siis koheldakse seda kui rööptüüpi ning selle muuttüübi loendur tõstetakse ühe võrra

Lisaks loendatakse muutumatuid sõnu ja muuttüüpi 0, mis on Vabamorfi loodud muuttüüp väga erandlike sõnade klassifitseerimiseks

Lõpus väljastatakse sagedustabel.

In [ ]:
print("---Mitme muuttüübiga sõnadel on oma kategooria---\n")

muuttüübid_freq = defaultdict(lambda: 0)
muutumatud_freq = 0

# Mitme muuttüübid omas kategoorias
for muuttüüp_list in muuttüübid:
  if not muuttüüp_list: # Muutumatud sõnad (muuttüüpi pole)
    muutumatud_freq += 1
  elif len(muuttüüp_list) == 1:
    muuttüübid_freq[str(muuttüüp_list[0])] += 1
  else:
    mttps = ""
    for i in range(len(muuttüüp_list)):
      item = muuttüüp_list[i]
      if i == len(muuttüüp_list) - 1:
        mttps += item
      else:
        mttps += item + ","
    muuttüübid_freq[mttps] += 1

muuttüübid_freq = sorted(muuttüübid_freq.items(), key=lambda item: item[1], reverse=True)

käändsõnad_freq = {}
pöördsõnad_freq = {}

for tüüp, freq in muuttüübid_freq:
  el = tüüp.split(",")
  #print(el, freq)
  if int(el[0]) == 0:
    tundmatu_freq = freq
  elif int(el[0]) <= 26:
    käändsõnad_freq[tüüp] = freq
  else:
    pöördsõnad_freq[tüüp] = freq

käändsõnad_sum = 0
pöördsõnad_sum = 0
summa_protsent = 0

print("Käändsõnad")
for item, freq in käändsõnad_freq.items():
  print(f"{item}: {freq} ({round(freq / len(muuttüübid) * 100, 4)}%)")
  käändsõnad_sum += freq
  summa_protsent += freq / len(muuttüübid) * 100
print(f"Kokku: {käändsõnad_sum} ({round(summa_protsent, 4)}%)\n")

summa_protsent = 0

print("Pöördsõnad")
for item, freq in pöördsõnad_freq.items():
  print(f"{item}: {freq} ({round(freq / len(muuttüübid) * 100, 4)}%)")
  pöördsõnad_sum += freq
  summa_protsent += freq / len(muuttüübid) * 100
print(f"Kokku: {pöördsõnad_sum} ({round(summa_protsent, 4)}%)\n")
print(f"Muuttüüp 0: {tundmatu_freq} ({round(tundmatu_freq / len(muuttüübid) * 100, 4)}%)")
print(f"Muutumatud sõnad: {muutumatud_freq} ({round(muutumatud_freq / len(muuttüübid) * 100, 4)}%)")
print(f"\nKokku: {käändsõnad_sum + pöördsõnad_sum + tundmatu_freq + muutumatud_freq} ({round((käändsõnad_sum + pöördsõnad_sum + tundmatu_freq + muutumatud_freq) / len(muuttüübid) * 100, 4)}%)")

---Mitme muuttüübiga sõnadel on oma kategooria---

Käändsõnad
22: 13941 (18.8828%)
2: 11260 (15.2515%)
1: 4640 (6.2848%)
12: 4286 (5.8053%)
11: 3890 (5.2689%)
16: 2722 (3.6869%)
11,9: 2462 (3.3347%)
17: 1721 (2.3311%)
25: 1514 (2.0507%)
26: 1256 (1.7012%)
10: 1085 (1.4696%)
6: 1063 (1.4398%)
12,10: 903 (1.2231%)
9: 586 (0.7937%)
19: 552 (0.7477%)
13: 531 (0.7192%)
7: 455 (0.6163%)
24: 399 (0.5404%)
18: 396 (0.5364%)
5: 227 (0.3075%)
23,22: 199 (0.2695%)
19,2: 187 (0.2533%)
14: 184 (0.2492%)
20: 108 (0.1463%)
4: 106 (0.1436%)
21: 99 (0.1341%)
8: 79 (0.107%)
15: 70 (0.0948%)
3,5: 25 (0.0339%)
23: 19 (0.0257%)
10,12: 14 (0.019%)
3: 8 (0.0108%)
5,7: 5 (0.0068%)
22,22: 1 (0.0014%)
Kokku: 54993 (74.487%)

Pöördsõnad
27: 5581 (7.5594%)
28: 2639 (3.5745%)
29: 722 (0.9779%)
36: 551 (0.7463%)
34: 463 (0.6271%)
30: 214 (0.2899%)
38: 169 (0.2289%)
37: 153 (0.2072%)
35: 112 (0.1517%)
31,27: 38 (0.0515%)
32: 37 (0.0501%)
28,27: 31 (0.042%)
33: 11 (0.0149%)
Kokku: 10721 (14.5214%)

Muuttüüp 0: 54 (0.

Järgmise koodiga analüüsitakse sõnu, täpsemalt keskmine sõna pikkus ja pikim sõna leksikonis.

In [ ]:
# Sõnade keskmine pikkus leksikonis
tähtede_arv = 0
sõnade_arv = ridade_arv

for sõna in sõnad:
  tähtede_arv += len(sõna[0]) # [0], sest sõna on järjendis

print(f"Keskmine sõna pikkus leksikonis: {round(tähtede_arv / sõnade_arv, 3)}")


# Pikim sõna
max_tähtede_arv = 0
pikim_sõna = ""

for sõna in sõnad:
  if len(sõna[0]) > max_tähtede_arv:
    pikim_sõna = sõna[0]
    max_tähtede_arv = len(sõna[0])

print(f"Pikim sõna leksikonis: {pikim_sõna} ({max_tähtede_arv} tähte)")

Keskmine sõna pikkus leksikonis: 9.046
Pikim sõna leksikonis: ratsionaliseerimisettepanek (27 tähte)


### Andmete salvestamine `.csv` faili

Selles peatükis salvestatakse kogutud andmed *.csv* faili.

In [ ]:
muuttüübi_0_algvormid = list()

In [ ]:
# Lemmad, muuttüüp (esialgu number, hiljem one-hot vektor vms.) .csv faili
veerud = ["Sõna", "Sõnaliik", "Muuttüüp"]
with open('muuttüübid_andmed.csv', 'w') as fail:
  csv_writer = csv.writer(fail)
  csv_writer.writerow(veerud)
  for sõna, sõnaliik, muuttüüp in zip(sõnad, sõnaliigid, muuttüübid):
    mttps = None
    sls = None
    # Muuttüübid
    if len(muuttüüp) == 0:
      mttps = "-"
    elif len(muuttüüp) == 1:
      if int(muuttüüp[0]) == 0: # Kui 0 muuttüüp, siis eira (lähevad eraldi)
        print(sõna, sõnaliik, muuttüüp)
        muuttüübi_0_algvormid.append(sõna)
        continue
    else:
      mttps = muuttüüp[0] + "," + muuttüüp[1]
    # Sõnaliigid
    if len(sõnaliik) == 0:
      sls = "-"
    elif len(sõnaliik) > 1:
      sls = sõnaliik[0] + "," + sõnaliik[1]
    if mttps and sls:
      csv_writer.writerow([sõna[0], sls, mttps])
    elif mttps:
      csv_writer.writerow([sõna[0], sõnaliik[0], mttps])
    elif sls:
      csv_writer.writerow([sõna[0], sls, str(muuttüüp[0])])
    else:
      csv_writer.writerow([sõna[0], sõnaliik[0], str(muuttüüp[0])])

In [ ]:
muuttüübi_0_algvormid[0][0]

In [ ]:
# 0 muuttüübiga salvestada andmed käändsõnad ja pöördsõnad eraldi faili
# igal real erivorm, sõnaliik, muuttüüp
sõnaliigi_regex = r'\\(\w{1,5})\\'
muuttüübi_regex = r'\.(\d{1,2})'

sõnad0 = []
sõnaliigid0 = []
muuttüübid0 = []
algvormid0 = []

keelud = ['$', '--']
lugeja = -1

with open('fs_lex.txt', 'r') as fs_lex:
# Andmete kättesaamine
  fs_lex.readline()
  for rida in fs_lex:
      #time.sleep(0.125)
      #display(rida)
      info = rida.split('|')[1].split('!')

      # Muuttüübi kättesaamine
      muuttüüp = re.findall(muuttüübi_regex, rida)

      # Sõnaliigi kättesaamine
      sõnaliik = re.findall(sõnaliigi_regex, rida)
      if sõnaliik:
        for i in range(len(sõnaliik)):
          sõnaliik[i] = sõnaliik[i][0]
          if sõnaliik[i] == 'B':
            sõnaliik[i] = 'A'

      if len(muuttüüp) == 1:
        if muuttüüp[0] == '0':
          lugeja += 1

          # Sõna kättesaamine
          sõnu = info[0].split(' ')

          # Iga sõna eraldi reale
          for s in sõnu:
            # Sõnade puhastamine
            puhastatud = re.sub(r'[\+\<\?\]\~\_\[\(\)\&\#\’\*\,]', '', s)
            if not puhastatud or puhastatud == '--':
              continue
            if '=' in puhastatud: # Mitmesõnaline
              puhastatud = puhastatud.replace('=', ' ')
            if puhastatud[0].isupper():
              puhastatud = puhastatud[0] + puhastatud[1:].lower()
            else:
              puhastatud = puhastatud.lower()

            sõnad0.append([puhastatud])
            sõnaliigid0.append(sõnaliik)
            muuttüübid0.append(muuttüüp)
            algvormid0.append(muuttüübi_0_algvormid[lugeja][0])


In [ ]:
# Salvestamine
veerud = ["Sõna", "Sõnaliik", "Muuttüüp", "Algvorm"]
with open('muuttüübi_0_andmed.csv', 'w') as fail:
  csv_writer = csv.writer(fail)
  csv_writer.writerow(veerud)
  for sõna, sõnaliik, muuttüüp, algvorm in zip(sõnad0, sõnaliigid0, muuttüübid0, algvormid0):
    mttps = None
    sls = None
    if len(muuttüüp) == 0:
      mttps = "-"
    if len(muuttüüp) > 1:
      mttps = muuttüüp[0] + "," + muuttüüp[1]
    if len(sõnaliik) > 1:
      sls = sõnaliik[0] + "," + sõnaliik[1]
    if mttps and sls:
      csv_writer.writerow([sõna[0], sls, mttps, algvorm])
    elif mttps:
      csv_writer.writerow([sõna[0], sõnaliik[0], mttps, algvorm])
    elif sls:
      csv_writer.writerow([sõna[0], sls, str(muuttüüp[0]), algvorm])
    else:
      csv_writer.writerow([sõna[0], sõnaliik[0], str(muuttüüp[0]), algvorm])